## Parameters

In [ ]:
# ------- PARAMETERS ------- #

RUNS = [155]      # run indexes

root = '/pnfs/pic.es/data/cta/LST/LST3/Data/ZFITS/' # path to all folders with the .zfits files

# -------------------------- #

# other parameters

dir_files     = 'csv_files/'  # path (relative or absolute) to the .csv files that will be created
only_subrun0  = True         # if we want to extract only the first subrun of a run

## Import packages

In [ ]:
import numpy  as np
import pandas as pd

from traitlets.config import Config
from ctapipe.io       import EventSource
from IPython.display  import clear_output

import auxiliar as aux

LST_camera = aux.find_LST_num(root) # extracting the number of LST

aux.create_folder(dir_files)        # creating the folder if dont exist

# Extract time of peak arrival in each pixel

In [ ]:
# over all runs
for run in RUNS:

    # finding the run in the folder root
    date, subruns = aux.search(root,run)    

    # to open a second run we need to use the first subrun of the run and
    # extract the timestamp of the first event

    # first subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,
            },
        })  

    path   = root + date + '/'
    path   = path + 'LST-' + LST_camera + '.1.Run' + str(run).zfill(5) + '.' + str(0).zfill(4) + '.fits.fz'
    source = EventSource(input_url=path, config=config, max_events=1)

    for i, ev in enumerate(source): 
        timestamp = int(ev.trigger.time.value)

    # following subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,

          "EventTimeCalculator": {
            "dragon_module_id":133 ,
            "dragon_reference_counter": 0,
            "dragon_reference_time": timestamp,
          },
            },
        })
    
    # over all subruns
    for subrun in subruns:
        
        # directory
        path   = root + date + '/'
        path   = path + 'LST-' + LST_camera + '.1.Run'+str(run).zfill(5)+'.'+str(subrun).zfill(4)+'.fits.fz'
        source = EventSource(input_url=path, config=config)


        # arrays 
        tPeak = [[] for i in range(1855)]
        time  =  []

        for i, ev in enumerate(source): 

            if i % 5000 == 0 or i == 0:
                print('Run ' + str(run) + ' Subrun ' + str(subrun) +' - '+str(round(100 * i / 53000,2)) + '%')
            
            # saving the time of arrival of the peak in each pixel
            for j in range(1855):
        
                tPeak[j].append(np.argmax(ev.r0.tel[1].waveform[0][j][5:]))
            time.append(ev.trigger.time.value)
            
        print('Run ' + str(run) + ' Subrun ' + str(subrun) + ' - 100%')
        print('\n creating the dataframe...')
        
        #we create the matrix for the dataframe
        matrix = [time]  
        for i in range(len(tPeak)):
            matrix.append(tPeak[i])

        del tPeak, time

        # creating the dataframe using pandas
        data = pd.DataFrame(np.transpose(matrix))

        del matrix

        # csv arxive
        data.to_csv(dir_files + 'data_tRes_Run' + str(run) + '_Subrun' + str(subrun) + '.csv', index=False)

        # clear memory and console
        clear_output()
        del data
        
print('Finished with the files creation')